In [1]:
import pandas as pd
import numpy as np

import re

In [2]:
train_data = pd.read_csv("champ1_train.csv", sep=',', encoding = "UTF-8")

## Описание ##

Регулярные выражения для выделения признаков:

1. `_IS_PRKNG_` Парковка. Значения: 
 - "нет", 
 - "городская парковка", 
 - "выделенная парковка".
 
 При выделении признака предполагаем, что:

 1. Выделенная парка имеет большую ценность, чем городская.
 2. Продавец благоразумен, т.е. при наличии выделенной парковки он упомянет это в описании, иначе просто опишет наличие парковки

 Таким образом, сначала все объявления где упоминается парковка -> обозначаем как 'городская парковка', далее проверяем упоминание про выделенную парковку и обновляем признак на 'выделенная парковка'. все остальные - 'нет'

2. `_IS_WIN_` Наличие витринных окон. Значения: 
 - 'нет', 
 - 'да'.

3. `_F1_H_` Высота потолков 1-го этажа

4. `_LINE_`  Линия расположения по отношению к улице, проспекту, переулку ит.п, где располагается помещение. Значения: 
 - 'первая линия', 
 - 'первая линия в глубине', 
 - 'внутри квартала'.

 Сейчас 'первая линия в глубине' == 'вторая линия'
 ЗАМЕЧАНИЕ: очень много пропусков
 
5. `_FOOT_TRAF_` Пешеходный трафик. Значения:  
 - 'низкий',  
 - 'средний',  
 - 'высокий',  
 - 'очень высокий'.

 Политика такая же как с парковками(см.п1) - о высоком трафике хозяин сообщает в объявлении.
 Может быть, имеет смысл брать эти данные не из объявления, а с геолокации

6. `_IS_SEP_ENT_` Наличие отдельного входа. Значения: 
 - 'нет', 
 - 'да'.
7. `_IS_VENT_` Наличие и тп системы центральной вентиляции и кондиционирования. Значения: 
 - 'сплит-системы',  
 - 'центральная вентиляция',  
 - 'центральные вентиляция и кондиционирования',  
 - 'нет'
 
8. `_DECOR_` Уровень отделки. Значения:  
 - 'рабочее состояние',  
 - 'евроремонт',  
 - 'без отделки'.
 
9. `_IS_COM_` Наличие коммуникаций. Значения:
 - 'все',  
 - 'имеются частично',  
 - 'отсутвуют'.
 
 Политика такая же как с парковками(см.п1) - о всех коммуникацияъ хозяин сообщает в объявлении.

## Шаблоны ##

In [3]:
patterns = {
    '_IS_PRKNG_': {
        u'выделенная парковка': ur'(выдел|своя|свое|собст|охран|персон|территор|подзем|огоро|личн)[\w ]*п(а|о)рк(овк|инг|овоч)[: \w]*',
        u'городская парковка': ur'([\w ]*п(а|о)рк(овк|инг|овоч)[: \w]*)|([\w ]*стоянк[: \w]*)'
    },
    '_IS_WIN_': {
        u'да': ur'(витраж|пан[ао]рамн)[\w]*|витр[\w]* (окн|осте|засте)[\w]*'
    },
    '_F1_H_': {
        u'number': ur'высот[ а-яА-Я]*[\-\:]*[ а-яА-Я]*[\d]+([.,\d]*)?([ ]*[мМ])?'
    },
    '_LINE_': {
        u'первая линия': ur'((перв[\w]+)|1) лин[\w]+',        
        u'первая линия в глубине': ur'((втор[\w]+)|2) лин[\w]+',
        u'внутри квартала': ur'(в глубине|внутри) квартала'
    },
    '_FOOT_TRAF_': {
        u'высокий': ur'трафик[\W]'
    },
    '_IS_SEP_ENT_': {
        u'да': ur'(отдел[\w]* вход[ом|а]?[\W])|(вход[ом|а]?\W[ \w]*отдельн[\w]*)'
    },
#     '_IS_VENT_': {
#         u'сплит-системы': ur'',
#         u'центральная вентиляция': ur'',
#         u'центральные вентиляция и кондиционирования': ur''
#     },
    '_DECOR_': { 
        u'евроремонт': ur'[\w]+вроремонт[\w]*',   
        u'без отделки': ur'(без|черно)[\w ]*отделк[\w :]+',
        u'рабочее состояние': ur'(отделк[\w :]+)|([\W]ремонт)'
    },
    '_IS_COM_': {
        'все': ur'(все|центр)[\w]* коммуника[\w]*',
        'имеются частично': ur'коммуника[\w]*'
    }
}

In [4]:
def getNumber(strr):
    p = re.compile(ur'[\d]+([.,\d]*)?([ ]*[mM])?', flags=re.UNICODE | re.IGNORECASE)
    groups = list(p.finditer(strr))
    return groups[-1].group() if len(groups) > 0 else strr

In [5]:
def finditer(desc, pattern_set):
    res = np.nan
#     print desc
    for pattern in pattern_set:
        p = re.compile(pattern_set[pattern], flags=re.UNICODE | re.IGNORECASE)
        groups = list(p.finditer(desc))
        if len(groups) > 0:
            res = pattern if pattern != u'number' else getNumber(groups[-1].group())
    return res

## Создание атрибутов ##

In [10]:
for pattern_set in patterns:
    print pattern_set
    train_data[pattern_set] = train_data['_DESC_'].apply(lambda x: finditer(x, patterns[pattern_set]))

_IS_COM_


C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: UnicodeWarning: Unicode unequal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


_DECOR_
_FOOT_TRAF_
_LINE_
_F1_H_
_IS_WIN_
_IS_PRKNG_
_IS_SEP_ENT_


## Сохранение ##

In [13]:
train_data.to_csv("champ1_train_ext.csv", sep=',', index=False, encoding = "UTF-8")